In [1]:
import numpy as np
import PIL.Image as image
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [2]:
# 加载图像，并对数据进行规范化
def load_data(filePath):
    # 读文件
    f = open(filePath,'rb')
    
    data = []
    
    # 得到图像的像素值
    img = image.open(f)
    
    # 得到图像尺寸
    width, height = img.size
    for x in range(width):
        for y in range(height):
            # 得到点(x,y)的三个通道值
            # 因为 jpg 格式的图像是三个通道 (R,G,B)，也就是一个像素点具有 3 个特征值。
            # 这里我们用 c1、c2、c3 来获取平面坐标点 (x,y) 的三个特征值，特征值是在 0-255 之间。
            c1, c2, c3 = img.getpixel((x, y))
            data.append([c1, c2, c3])
    f.close()
    
    # 采用Min-Max规范化
    # 为了加快聚类的收敛，我们需要采用 Min-Max 规范化对数据进行规范化。
    mm = preprocessing.MinMaxScaler()
    data = mm.fit_transform(data)
    return np.mat(data), width, height

In [3]:
# 加载图像，得到规范化的结果img，以及图像尺寸
img, width, height = load_data('weixin.jpg')

In [5]:
print(img)
print(width)
print(height)

[[0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 ...
 [0.00865801 0.04310345 0.08606557]
 [0.00865801 0.04310345 0.08606557]
 [0.00865801 0.04310345 0.08606557]]
100
177


In [6]:
# 用K-Means对图像进行2聚类
kmeans =KMeans(n_clusters=2)
kmeans.fit(img)
label = kmeans.predict(img)

In [7]:
# 将图像聚类结果，转化成图像尺寸的矩阵
label = label.reshape([width, height])

In [8]:
# 如果你想对图像聚类的结果进行可视化，直接看0和1是看不出来的，还需要将0和1转化为灰度值。
# 灰度值一般是在0-255的范围内，我们可以将label=0设定为灰度值255，label=1设定为灰度值127。
# 具体方法是用 int(256/(label[x][y]+1))-1。
# 可视化的时候，主要是通过设置图像的灰度值进行显示。
# 所以我们把聚类label=0的像素点都统一设置灰度值为 255，把聚类label=1的像素点都统一设置灰度值为127。
# 原来图像的灰度值是在0-255之间，现在就只有2种颜色（也就是灰度为255，和灰度127）。

# 创建个新图像pic_mark，用来保存图像聚类的结果，并设置不同的灰度值
pic_mark = image.new("L", (width, height))
for x in range(width):
    for y in range(height):
        # 根据类别设置图像灰度, 类别0 灰度值为255， 类别1 灰度值为127
        pic_mark.putpixel((x, y), int(256/(label[x][y]+1))-1)
pic_mark.save("weixin_mark.jpg", "JPEG")

In [9]:
# 用K-Means对图像进行16聚类
kmeans =KMeans(n_clusters=16)
kmeans.fit(img)
label = kmeans.predict(img)

In [10]:
# 将图像聚类结果，转化成图像尺寸的矩阵
label = label.reshape([width, height])

In [11]:
print(label)

[[4 4 4 ... 9 9 9]
 [4 4 4 ... 9 9 9]
 [4 4 4 ... 9 9 9]
 ...
 [4 4 4 ... 9 9 9]
 [4 4 4 ... 9 9 9]
 [4 4 4 ... 9 9 9]]


In [16]:
from skimage import color

# 使用 skimage 中的 label2rgb 函数来将 label 分类标识转化为颜色数值，因为我们的颜色值范围是 [0,255]，所以还需要乘以 255 进行转化，最后再转化为 np.uint8
# 类型。unit8 类型代表无符号整数，范围是 0-255 之间。
# 将聚类标识矩阵转化为不同颜色的矩阵
label_color = (color.label2rgb(label)*255).astype(np.uint8)

# 得到颜色矩阵后，你可以把它输出出来，这时你发现输出的图像是颠倒的，原因可能是图像源拍摄的时候本身是倒置的。
# 我们需要设置三维矩阵的转置，让第一维和第二维颠倒过来，也就是使用 transpose(1,0,2)，将原来的 (0,1,2）顺序转化为 (1,0,2) 顺序，即第一维和第二维互换。
label_color = label_color.transpose(1,0,2)

# 使用 fromarray 函数，它可以通过矩阵来生成图片，并使用 save 进行保存。
images = image.fromarray(label_color)
images.save('weixin_mark_color.jpg')

ImportError: cannot import name 'img_as_float32'

In [17]:
# 创建个新图像img，用来保存图像聚类压缩后的结果
img=image.new('RGB', (width, height))
for x in range(width):
    for y in range(height):
        c1 = kmeans.cluster_centers_[label[x, y], 0]
        c2 = kmeans.cluster_centers_[label[x, y], 1]
        c3 = kmeans.cluster_centers_[label[x, y], 2]
        img.putpixel((x, y), (int(c1*256)-1, int(c2*256)-1, int(c3*256)-1))
img.save('weixin_new.jpg')